In [1]:
from datasets import load_dataset, Dataset
from ragas import evaluate
from ragas.metrics import context_recall, context_precision, answer_correctness
from src.retriever import get_relevant_chunks
import os, itertools

d:\projects\RagGpt\src\retriever.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  _embeddings = HuggingFaceEmbeddings(model_name=settings.EMBEDDING_MODEL)


In [2]:
ds = load_dataset("AndrewTsai0406/RGB_zh",split="train")
#hotpot = hotpot.filter(lambda x: x["answer"].strip().lower() not in ["yes","no"])

In [3]:
ds[2]["query"]

'第七届香港立法会议员宣誓仪式的时间'

In [4]:
questions = ds["query"]
ground_truths = ds["answer"]       
contexts_block = [
    pos + neg                 # 把当前样本中的positive和negative两段列表拼起来
    for pos, neg in zip(ds["positive"], ds["negative"])
]
  

In [5]:
import ast   # 用 ast.literal_eval 比 eval 安全

def clean_answer(ans):
    """
    把像 "['12人']" 这样的字符串 → '12人'
    如果本来就是普通字符串，则原样返回
    """
    if isinstance(ans, str) and ans.startswith('[') and ans.endswith(']'):
        try:
            return ast.literal_eval(ans)[0]   # 解析成列表后取第 0 个元素
        except Exception:
            return ans
    return ans

# ds["answer"] 是 list，用列表推导逐个清洗
ground_truths = [clean_answer(a) for a in ds["answer"]]

In [6]:
print(ground_truths[99])

中国


In [ ]:
contexts_block[2]

In [ ]:
len(contexts_block)

In [ ]:
type(contexts_block)

In [ ]:
#save_dir = './documents/RGB_zh'
os.makedirs(save_dir, exist_ok=True)   

# ----------- 写入文件 -----------
for idx, item in enumerate(contexts_block, start=1):
    # 生成文件名：001.txt、002.txt……
    filename = os.path.join(save_dir, f'{idx:03d}.txt')
    
    # 如果 item 不是字符串，先转成字符串；如需做清洗可在这里处理
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(str(item))

print(f' 已写入 {len(contexts_block)} 个文件到 {save_dir}')


In [14]:
contexts_list, manual_hits = [], []

for i, (q, gt) in enumerate(zip(questions, ground_truths)):
    ctxs = get_relevant_chunks(q, k=3)          # 召回的段落列表
    contexts_list.append(ctxs)

    # 把 gt 统一成 list，方便后面双层 any
    answers = gt if isinstance(gt, list) else [gt]

    # 只要某个答案出现在任意 ctx 里，就算命中
    hit = any(ans in c for ans in answers for c in ctxs)
    manual_hits.append(hit)

    print(f"[{i:02d}] hit={hit}  retrieved={len(ctxs)}\n\n")

print(f"\nManual Recall: {sum(manual_hits)/len(manual_hits):.3f}")


[Retriever] Intersection (交集) count: 24
[00] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[01] hit=False  retrieved=3


[Retriever] Intersection (交集) count: 0
[02] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[03] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[04] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[05] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[06] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[07] hit=False  retrieved=3


[Retriever] Intersection (交集) count: 0
[08] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[09] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[10] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[11] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[12] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[13] hit=True  retrieved=3


[Retriever] Intersection (交集) count: 0
[14] h

In [11]:
answers = [" ".join(c) for c in contexts_list]  # 或替换为真实的 LLM 生成

In [12]:
#  查看第n个样例
n = 299
print(f"--- Sample {n} ---")
print("Q :", questions[n])
print("GT:", ground_truths[n])
print("Ctxs:")
for j, c in enumerate(contexts_list[n]):
    print(f"  ({j})", c)
print("\nAns:", answers[n])

--- Sample 299 ---
Q : 2023年温网女单亚军
GT: 贾巴尔
Ctxs:
  (0) 作为去年打进温网女单决赛的两名姑娘，冠军莱巴金娜和亚军贾巴尔，一个\xa0...', 'Jul 8, 2023 ...
  (1) 'Jul 8, 2023 ... 今晨，2023年温网结束1场女单焦点战。中国金花白卓璇结束神奇旅程，第2轮只抵抗了45分钟就以2个1-6速败上届亚军贾巴尔，未能更进一步。中国女单温网\xa0...']['2023年温布尔登网球锦标赛女子单打比赛，是2023年温布尔登网球锦标赛的其中一个比赛项目。伊莲娜·莱巴金娜是卫冕冠军，[1]但她在1/4决赛中输给本届亚军昂丝·加博。
  (2) 'Jul 17, 2023 ... 北京时间7月15日，2023年温网女单决赛，上届亚军、6号种子贾巴尔对阵非种子选手万卓索娃。万卓索娃直落两盘总分2-0横扫贾巴尔，勇夺温网女单冠军，\xa0...', 'Apr 3, 2023 ... 这张看似极为用心的温网海报设计，恰恰因为用心过度，才引发了舆论的强烈不满。 作为去年打进温网女单决赛的两名姑娘，冠军莱巴金娜和亚军贾巴尔，一个\xa0...',
  (3) 这张看似极为用心的温网海报设计，恰恰因为用心过度，才引发了舆论的强烈不满。  作为去年打进温网女单决赛的两名姑娘，冠军莱巴金娜和亚军贾巴尔，一个占据C位，一个位居次C位。', 'Jul 15, 2023 ... 央视网消息：北京时间7月15日，2023年温网女单决赛，上届亚军、6号种子贾巴尔对阵非种子选手万卓索娃。万卓索娃直落两盘总分2-0横扫贾巴尔，勇夺温网\xa0...', 'Jul
  (4) @Wimbledon 事实上2023年温网男单决赛的第一盘是由乔科维奇以以6：1拿下，在这看似薑是老的辣的开局表现后，现任球王反而凭藉自己的抗压性与不断进步的球技连追2盘逆转听牌，但是乔科维奇随之在第4盘又用6：3的比数替比赛带来精彩悬念。', '21世纪经济报道记者彭强 北京报道

Ans: 作为去年打进温网女单决赛的两名姑娘，冠军莱巴金娜和亚军贾巴尔，一个\xa0...', 'Jul 8, 2023 ... 'Jul 8, 2023 ... 今晨，2023年温网结束1场女单焦点战。中国金花白卓璇结束神奇旅程，第2轮只抵抗了45分钟就以2个1-6

sample0是一个典型的 2-Hop 问答。
我们检索到了多条 “Kiss and Tell / Meet Corliss Archer”电影和角色有关的信息，
但缺少了 Shirley Temple 的政府职务。
要答对，必须先命中 Shirley Temple 的人物条目，再从条目里捞到她的政府履历。检索必须跨文档联结电影 → 人物 → 职位。

解决：
1.改写query，多路召回，比如LangChain MultiQueryRetriever
2.agent rag，让 Agent 边检索边思考：Step-1 找演员 -》 Step-2 找职位 -》 汇总
2.Graph，把文档主动转成“实体-关系图”再检索，适用于多跳。目前只有一跳，不一定要用。且查询成本很大。


In [ ]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from config import settings
#  组装 RAGAS 数据并评估

data = {
    "question":     questions,
    "contexts":     contexts_list,
    #"answer":       answers,
    "ground_truth": ground_truths,
}
print("Data lengths:", {k: len(v) for k, v in data.items()})

eval_ds = Dataset.from_dict(data)
scores  = evaluate(
    dataset=eval_ds,
    metrics=[context_recall, context_precision],
    llm = LangchainLLMWrapper(ChatOpenAI(model=settings.CHAT_MODEL))
)
df = scores.to_pandas()

print("\n=== RAGAS 评估结果 ===")
print(df)

avg = df[["context_recall","context_precision"]].mean()
print("\n=== 平均指标 ===")
print(avg.to_string())